# Assembly of the *Hippodamia convergens* genome

This workflow outlines the process of assembling the *Hippodamia convergens* genome by utilising genomic data from 20 unsexed beetles (metagenomic data). The genomic data consists of continuous long reads (CLRs) that were generated using the PacBio RS II sequencing platform, resulting in a total of 265 gigabases (Gb) of genomic data. Several genome assembly tools (*Canu*, *wtdbg2*, and *Flye*) were employed to determine which one generates the highest-quality primary assembly. Assembly quality will be based on the 3C criterion of completion, correctness, and contiguity.

## Step 1: Download genomic data and save as environmental variables

In [ ]:
# Step 1.1: Create a directory for all assembly tool outputs
mkdir pipeline1
cd pipeline1 

# Step 1.2 Concatenate pacbio long reads
cat genomic_data/XDOVE_20201125_S64049_PL100162351-1_B01.subreads.fastq \
genomic_data/SM01-DTG-DNA-566_c1-Cell1.subreads.bam.fastq \
genomic_data/SM01-DTG-DNA-566_c2-Cell1.subreads.bam.fastq > pacbio_all.fastq

rm -rf XDOVE_20201125_S64049_PL100162351-1_B01.subreads.fastq \
SM01-DTG-DNA-566_c1-Cell1.subreads.bam.fastq \
SM01-DTG-DNA-566_c2-Cell1.subreads.bam.fastq \

# Step 1.3: Move all genomic data into a folder
mkdir genomic_data
mv pacbio_all.fastq genomic_data
mv haxyr.fa genomic_data

# Step 1.4: Set genomic data as environmental variables
export pacbio_all="/home2/gavrila/work/hcon_assembly/genomic_data/pacbio_all.fastq" 
export haxyr="/home2/gavrila/work/hcon_assembly/genomic_data/haxyr.fa"

## Step 2: Export tool paths

In [ ]:
# Step 2.1: Set canu path
export PATH=$PATH:/home2/gavrila/work/tools/canu-2.2/bin

# Step 2.2: Set purge_dups path
export PATH=$PATH:/home2/gavrila/work/tools/purge_dups/bin

# Step 2.3: Set minimap2 path
export PATH=$PATH:/home2/gavrila/work/tools/minimap2

# Step 2.4: Set filtlong path 
export PATH=$PATH:/home2/gavrila/work/tools/Filtlong/bin 

# Step 2.5: Set ragtag path
export PATH=$PATH:/home2/gavrila/work/tools/ragtag/bin

# Step 2.6: Set Augustus path
export PATH=$PATH:/home2/gavrila/work/tools/Augustus/bin

# Step 2.7: Set AUGUSTUS-helpers path
export PATH=$PATH:/home2/gavrila/work/tools/AUGUSTUS-helpers
    
# Step 2.8: Set wtdbg2 path
export PATH=$PATH:/home2/gavrila/work/tools/wtdbg2
    
# Step 2.9: Set Flye path
export PATH=$PATH:/home2/gavrila/work/tools/Flye/bin
    
# Step 2.10: Set samtools path
export PATH=$PATH:/home2/gavrila/work/tools/samtools-1.16.1

## Step 3: Data Preprocessing

Prior to genome assembly, the raw sequencing data needs to be preprocessed to remove any low-quality reads, adapters, or contaminants. This step is essential for obtaining accurate and reliable results. Both PacBio CLR reads and Illumina Hi-C paired-end reads have already been processed by DoveTail genomics for the removal of adapters and low-quality reads. All genomic reads are ready for assembly.


## Step 4a : Assembly with *Canu*

In [ ]:
# Step 4a.1: Assembly with Canu
canu \
 -p canu_assembly -d canu_output \
 genomeSize=450m \
 -pacbio $pacbio_all
echo "Step 4a.1 complete" &&

# Step 4a.2: BUSCO Check
busco -i canu_output/canu_assembly.contigs.fasta  -l endopterygota_odb10 -o canu_output/busco_output -m genome -c 48 &&
echo "Step 4a.2 complete" &&

# Step 4a.3: QUAST Check
quast canu_output/canu_assembly.contigs.fasta -o canu_output/quast_output --split-scaffolds &&
echo "Step 4a.3 complete" &&

## Step 4b : Assembly with *wtdbg2*

In [ ]:
# Step 4b.1: Create an output folder for purge_dup generated files
mkdir wtdbg2_output
cd wtdbg2_output
echo "Step 4b.1 complete" &&

# Step 4b.2: Assembly with wtdbg2
wtdbg2 -x sq -L 20000 -l 8192 -R -g 450m -i $pacbio_all -t 48 -fo assembly &&
echo "Step 4b.2 complete" &&

# Step 4b.3: Derive consensus
wtpoa-cns -t 48 -i assembly.ctg.lay.gz -fo assembly.raw.fa &&
echo "Step 4b.3 complete" &&

# Step 4b.4: Polish consensus using long reads
minimap2 -t 48 -ax map-pb -r2k assembly.raw.fa $pacbio_all | samtools sort -@4 > assembly.bam &&
samtools view -F0x900 assembly.bam | ./wtpoa-cns -t 48 -d assembly.raw.fa -i - -fo assembly.cns.fa &&
echo "Step 4b.4 complete" &&

# Step 4b.5: Move out of the wtdbg2 folder
cd ..

## Step 4c : Assembly with *Flye*

In [ ]:
# Step 4c: Assembly with Flye
flye --pacbio-raw $pacbio_all --out-dir flye_output --threads 48 --genome-size 450m --meta --no-alt-contigs &&
echo "Step 4c complete" &&

## Step 5: Assembly Duplication Removal

NOTE: The subsequent script was executed for every primary genome assembly. For the sake of brevity, the respective purge_dup scripts for wtdbg2 and Flye were not incorporated into this workflow.

In [ ]:
# Step 5.1: Create an output folder for purge_dup generated files
mkdir purgedup_output
cd purgedup_output
echo "Step 5.1 complete" &&

# Step 5.2: Run minimap2 to align pacbio data agains the assembly and generate paf files, 
minimap2 -t48 -xmap-pb ../canu_output/canu_assembly.fasta \
/userdata/gavrila/XDOVE_20201125_S64049_PL100162351-1_B01.subreads.fastq | gzip -c - > pb1.paf.gz &&
echo "Step 5.2 complete" &&

# Step 5.3: Then run the pbcstat, calcuts functions on it to calculate read depth histogram and base-level read depth
pbcstat *.paf.gz &&
calcuts PB.stat > cutoffs 2> calcults.log && 
echo "Step 5.3 complete" &&

# Step 5.4: Then split the assembly file, do a self alignment
split_fa ../canu_output/canu_assembly.fasta > masked_genome.split &&
echo "Step 5.4 complete" &&

# Step 5.5: Run minimap2 for self alignment
minimap2 -t48 -xasm5 -DP masked_genome.split masked_genome.split | gzip -c - > masked_genome.split.self.paf.gz &&
echo "Step 5.5 complete" &&

# Step 5.6: Purge haplotigs and overlaps
purge_dups -2 -T cutoffs -c PB.base.cov masked_genome.split.self.paf.gz > dups.bed 2> purge_dups.log &&
echo "Step 5.6 complete" &&

# Step 5.7: Get purged primary and haplotig sequences from draft assembly
get_seqs -e dups.bed ../canu_output/canu_assembly.fasta &&
echo "Step 5.7 complete" &&

# Step 5.8: BUSCO Check
busco -i purged.fa -l endopterygota_odb10 -o busco_output -m genome -c 48 &&
echo "Step 5.8 complete" &&

# Step 5.9: QUAST Check
quast purged.fa -o quast_output --split-scaffolds &&
echo "Step 5.9 complete" &&

# Step 5.10: Move out of the purgedup folder
cd ..

Out of all the purged primary assemblies, the purged *Canu* assembly retained the highest BUSCO completion score. Therefore, it was selected for additional polishing.

## Step 6: Assembly Scaffolding

In [ ]:
# Step 6: Run RagTag scaffolding using the harmonia axyridis genome as a reference
ragtag.py scaffold $haxyr purgedup_output/purged.fa -o ragtag_scaffold_output &&
echo "Step 6 complete" &&

## Step 7: Assembly Filtering

In [ ]:
# Step 7.1: Create an output folder for ragtag generated files
mkdir ragtag_scaffold_output
echo "Step 7.1 complete" &&

# Step 7.2: Remove all contigs less than 50000 bases long from the scaffolded assembly
filtlong --min_length 50000 ragtag_scaffold_output/ragtag.scaffold.fasta > filtlong_output/assembly.fasta &&
echo "Step 7.2 complete" &&

# Step 7.3: Move out of the ragtag folder
cd ..

## Step 8: Final Assembly Quality Assessment

In [ ]:
# Step 8.1: Run BUSCO analysis
busco -i filtlong_output/assembly.fasta -l endopterygota_odb10 -o ragtag_scaffold_output/busco_output -m genome -c 48 &&
echo "Step 8.1 complete" &&

# Step 8.2: Run QUAST analysis
quast filtlong_output/assembly.fasta -o ragtag_scaffold_output/quast_output --split-scaffolds &&
echo "Step 8.2 complete" &&

In [ ]:
echo "Assembly workflow complete"

## Conclusion
A genome assembly from the sequences of 20 unsexed H. convergens beetles is presented. The genome sequence is 615 megabases in span. The assembly was evaluated using the 3C criterion (completeness, contiguity, and correctness), and achieved a scaffold N50 length of 89 Mb, indicating good contiguity and an overall BUSCO score of 96.1% indicated a high level of completeness. However, 17% of the BUSCOs were found to be duplicated, suggesting room for improvement in genome correctness. 